In [ ]:
from stardist.models import StarDist2D

# prints a list of available models
StarDist2D.from_pretrained()

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

In [ ]:
from stardist.models import StarDist2D  # Import the StarDist2D model if not already done
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Load your PNG image
img_path = '/rsrch5/home/plm/yshokrollahi/project4/PUC_PATCH_1.png'  # Update this path to your PNG image's location
img = Image.open(img_path)

# If your model expects grayscale images, convert the image to grayscale
img = img.convert('L')  # This also ignores any alpha channel by converting to grayscale

img = np.array(img)

# Assuming 'model' is an instance of StarDist2D and has been properly loaded or trained
labels, _ = model.predict_instances(normalize(img))

# Plotting
plt.subplot(1,2,1)
plt.imshow(img, cmap="gray")
plt.axis("off")
plt.title("Input Image")

plt.subplot(1,2,2)
plt.imshow(render_label(labels, img=img))
plt.axis("off")
plt.title("Prediction + Input Overlay")

plt.show()


In [ ]:
from stardist.models import StarDist2D  # Import the StarDist2D model if not already done
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Load your image
img_path = '/rsrch5/home/plm/yshokrollahi/project4/18_roi_845.jpg'  # Update this path to your image's location
img = Image.open(img_path)

# Extract only the blue channel
# In a color image loaded by PIL, the channels are in the order (R, G, B)
# The blue channel is at index 2
blue_channel = np.array(img)[:,:,2]

# Now `blue_channel` is just the blue component of your image
img = blue_channel

# Assuming 'model' is an instance of StarDist2D and has been properly loaded or trained
labels, _ = model.predict_instances(normalize(img))

# Plotting
plt.subplot(1,2,1)
plt.imshow(img, cmap="gray")  # Even though it's the blue channel, we display it in grayscale
plt.axis("off")
plt.title("Input Image (Blue Channel)")

plt.subplot(1,2,2)
plt.imshow(render_label(labels, img=img))
plt.axis("off")
plt.title("Prediction + Input Overlay")

plt.show()

# Assuming 'labels' is the output from model.predict_instances(normalize(img))

# Count the unique labels, ignoring the background (label 0)
unique_labels = np.unique(labels)
cell_count = len(unique_labels) - 1  # Subtract one to ignore the background label

print(f"Detected cell count: {cell_count}")

# Your existing plotting code here for visualization



In [ ]:
import os
from PIL import Image, UnidentifiedImageError
import numpy as np
import matplotlib.pyplot as plt
from stardist.models import StarDist2D
from stardist.plot import render_label
from csbdeep.utils import normalize

# Function to load and predict a single image
def predict_and_count_with_centroids(model, img_path):
    try:
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')  # Ensure the image is in RGB mode
        img_np = np.array(img)
        blue_channel = img_np[:,:,2]  # Select the blue channel
        
        labels, details = model.predict_instances(normalize(blue_channel))
        
        unique_labels = np.unique(labels)
        predicted_count = len(unique_labels) - 1
        
        centroids = details['points']
        
        return labels, predicted_count, centroids
    except UnidentifiedImageError:
        print(f"Cannot identify image file {img_path}. Skipping...")
        return None, None, None
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None, None, None

# Load the pretrained StarDist model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

# Directories containing the validation images and their labels
img_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/images'
label_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/labels'

# Get a list of image files in the image directory, ignoring hidden files and focusing on PNG format
img_files = [f for f in os.listdir(img_folder) if f.endswith('.png') and not f.startswith('.')]

# Results list to hold prediction and ground truth counts
results = []

# Process each image
for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    
    # Predict cell count
    labels, predicted_count = predict_and_count(model, img_path)
    
    if labels is not None and predicted_count is not None:
        # Read the ground truth cell count
        label_file_path = os.path.join(label_folder, img_file.replace('.png', '.txt'))
        with open(label_file_path, 'r') as label_file:
            ground_truth_count = len(label_file.readlines())
        
        # Store the results
        results.append((img_file, predicted_count, ground_truth_count))

        # Optional: Display the overlay of prediction on the input image for verification
        plt.imshow(render_label(labels, img=np.array(Image.open(img_path).convert('L'))))
        plt.axis("off")
        plt.title(f"Prediction + Input Overlay\n{img_file}")
        plt.show()
        
        print(f"File: {img_file}, Predicted: {predicted_count}, Ground Truth: {ground_truth_count}")
    else:
        print(f"Skipping {img_file} due to errors.")

# Summary statistics or further analysis can be done here based on the `results` list


In [ ]:
import pandas as pd

# Continue from the previous code
# Assuming `results` list is populated with (img_file, predicted_count, ground_truth_count)

# Convert results into a pandas DataFrame
df_results = pd.DataFrame(results, columns=['Image File', 'Predicted Count', 'Ground Truth Count'])

# Calculate the difference between predicted and ground truth counts
df_results['Difference'] = df_results['Predicted Count'] - df_results['Ground Truth Count']

# Display the table of results
print("\nTable of Results:")
print(df_results)

# Calculate and display some basic statistics
summary_stats = df_results.describe()
print("\nSummary Statistics:")
print(summary_stats)

# Optional: Save the results table and statistics to CSV files
df_results.to_csv('/rsrch5/home/plm/yshokrollahi/project4/val_pack/results_table.csv', index=False)
summary_stats.to_csv('/rsrch5/home/plm/yshokrollahi/project4/val_pack/summary_statistics.csv')

print("\nResults and summary statistics have been saved to CSV files.")


In [ ]:
import os
from PIL import Image, UnidentifiedImageError
import numpy as np
import matplotlib.pyplot as plt
from stardist.models import StarDist2D
from csbdeep.utils import normalize

def predict_and_count_with_centroids(model, img_path):
    try:
        img = Image.open(img_path)
        img = img.convert('L')  # Convert to grayscale
        img_np = np.array(img)
        
        labels, details = model.predict_instances(normalize(img_np))
        
        unique_labels = np.unique(labels)
        predicted_count = len(unique_labels) - 1
        
        centroids = details['points']
        
        return labels, predicted_count, centroids
    except UnidentifiedImageError:
        print(f"Cannot identify image file {img_path}. Skipping...")
        return None, None, None
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None, None, None

model_name = '2D_versatile_fluo'
model = StarDist2D.from_pretrained(model_name)

img_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/images'
label_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/labels'
label_pred_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/labels_predict'  # Ensure this directory exists

if not os.path.exists(label_pred_folder):
    os.makedirs(label_pred_folder)

img_files = [f for f in os.listdir(img_folder) if f.endswith('.png') and not f.startswith('.')]

results = []

for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    
    labels, predicted_count, centroids = predict_and_count_with_centroids(model, img_path)
    
    if labels is not None and predicted_count is not None:
        label_file_path = os.path.join(label_folder, img_file.replace('.png', '.txt'))
        with open(label_file_path, 'r') as label_file:
            ground_truth_count = len(label_file.readlines())
        
        results.append((img_file, predicted_count, ground_truth_count, centroids))
        
        # Saving centroids to text file
        pred_label_file_path = os.path.join(label_pred_folder, img_file.replace('.png', '_pred.txt'))
        with open(pred_label_file_path, 'w') as f:
            for centroid in centroids:
                f.write(f"{centroid[0]}, {centroid[1]}\n")

        print(f"File: {img_file}, Predicted: {predicted_count}, Ground Truth: {ground_truth_count}, Centroids: {len(centroids)}")
    else:
        print(f"Skipping {img_file} due to errors.")


In [ ]:
import os
from PIL import Image, UnidentifiedImageError
import numpy as np
import matplotlib.pyplot as plt
from stardist.models import StarDist2D
from csbdeep.utils import normalize

def predict_and_count_with_centroids(model, img_path):
    try:
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')  # Ensure the image is in RGB mode
        img_np = np.array(img)
        blue_channel = img_np[:,:,2]  # Select the blue channel
        
        labels, details = model.predict_instances(normalize(blue_channel))
        
        unique_labels = np.unique(labels)
        predicted_count = len(unique_labels) - 1
        
        centroids = details['points']
        
        return labels, predicted_count, centroids
    except UnidentifiedImageError:
        print(f"Cannot identify image file {img_path}. Skipping...")
        return None, None, None
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None, None, None


model_name = '2D_paper_dsb2018'
model = StarDist2D.from_pretrained(model_name)

img_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/images'
label_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/labels'
label_pred_folder = '/rsrch5/home/plm/yshokrollahi/project4/val_pack/labels_predict'  # Ensure this directory exists

if not os.path.exists(label_pred_folder):
    os.makedirs(label_pred_folder)

img_files = [f for f in os.listdir(img_folder) if f.endswith('.png') and not f.startswith('.')]

results = []

for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    
    labels, predicted_count, centroids = predict_and_count_with_centroids(model, img_path)
    
    if labels is not None and predicted_count is not None:
        label_file_path = os.path.join(label_folder, img_file.replace('.png', '.txt'))
        with open(label_file_path, 'r') as label_file:
            ground_truth_count = len(label_file.readlines())
        
        results.append((img_file, predicted_count, ground_truth_count, centroids))
        
        # Saving centroids to text file
        pred_label_file_path = os.path.join(label_pred_folder, img_file.replace('.png', '_pred.txt'))
        with open(pred_label_file_path, 'w') as f:
            for centroid in centroids:
                f.write(f"{centroid[0]}, {centroid[1]}\n")

        print(f"File: {img_file}, Predicted: {predicted_count}, Ground Truth: {ground_truth_count}, Centroids: {len(centroids)}")
    else:
        print(f"Skipping {img_file} due to errors.")
